### Business Understanding 

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

### Business Objectives

1. **Identify Top-Performing Genres and Themes**:
   - Analyze box office trends to determine which genres (e.g., action, comedy, drama) and themes (e.g., superhero stories, family-friendly content) are consistently performing well. This will help prioritize the types of films to focus on.

2. **Understand Audience Demographics and Preferences**:
   - Use data to uncover insights about the target audience, such as age groups, gender, and regional preferences. For example, are younger audiences gravitating toward animated films, or are thrillers more popular among older viewers?

3. **Evaluate Seasonal and Regional Trends**:
   - Study how box office performance varies by season (e.g., summer blockbusters, holiday releases) and region. This can help in planning release schedules and tailoring content to specific markets.

### Data Understanding 

### Project Plan: Exploring Top-Performing Film Trends for a New Movie Studio

**Overview/Background:**  
The entertainment landscape is rapidly evolving, with data-driven strategies becoming crucial to creating successful films. Your company’s decision to establish a movie studio provides an exciting opportunity to explore and capitalize on current trends dominating the box office. By understanding what audiences want—be it genres, themes, or release timing—your studio can ensure its films align with market demand. Leveraging web-scraped data from box office records, social media trends, and streaming platforms can provide actionable insights for strategic decision-making.

**Challenges:**  
One of the main challenges involves analyzing the vast volume of unstructured data collected from multiple sources. Sorting through box office trends, audience demographics, and seasonal performance patterns while ensuring data accuracy and reliability may present difficulties. Additionally, translating data insights into creative decisions without stifling artistic freedom will require a careful balance. Legal and ethical considerations, such as respecting intellectual property rights and ensuring data compliance, also remain critical.

**Proposed Solution:**  
To address these challenges, the project will implement a structured data analytics framework. Using Python and libraries like pandas for data preprocessing, statistical models will identify top-performing genres, themes, and audience preferences. Data visualization tools such as Matplotlib or Tableau will help translate insights into clear, actionable visuals for decision-makers. Machine learning algorithms can assist in predicting future trends, ensuring the studio stays ahead of the curve. Regular cross-functional meetings between creative and analytics teams will help balance data-driven strategy with artistic vision.

**Brief Conclusion:**  
By prioritizing data analysis and predictive modeling, the studio can make informed decisions about the types of films to produce. This data-driven approach will minimize risks and maximize box office success. With creativity and analytics working hand-in-hand, the movie studio can establish itself as a leader in the competitive film industry.

### Loading the Data

In [2]:
# Importing the necessary Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# Loading the data from the different datasets
box_office_mojo = pd.read_csv("zippedData/bom.movie_gross.csv.gz", compression='gzip')
box_office_mojo.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


As you can see from the output above, this dataset contains columns of information on different movies including columns for domestic and foreign gross of the movies. This columns will be the focus as it will answer some of the business questions during analysis. 

In [4]:
# Loading data from imdb(sqlite3 database file)
import sqlite3

# Path to the unzipped .db file
db_path = 'unzippedData/im.db'

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Example: List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

# # Close the connection
# conn.close()

Tables in the database: [('movie_basics',), ('directors',), ('known_for',), ('movie_akas',), ('movie_ratings',), ('persons',), ('principals',), ('writers',)]


As you can see from the output above, the sqlite3 database file contains tables with different information about movies. Some of the tables will be used to answer some of our business questions. They include: ```"movie_basics"``` and ```"movie_ratings"```.

In [14]:
# Query the 'movie_ratings' table using pandas
movie_ratings_df = pd.read_sql_query("SELECT * FROM movie_ratings;", conn)
movie_ratings_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [15]:
# Query the 'movie_basics' table using pandas
movie_basics_df = pd.read_sql_query("SELECT * FROM movie_basics;", conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [16]:
# Merging movie_ratings_df and movie_basics_df on the 'movie_id' column
ratings_basics_merged = movie_ratings_df.merge(movie_basics_df, on="movie_id", how="inner")

# Display the first few rows of the merged DataFrame
ratings_basics_merged.head()

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"


In [17]:
# Create a new DataFrame with selected columns
ratings_summary_df = ratings_basics_merged[['averagerating', 'numvotes', 'primary_title']]

# Display the first few rows of the new DataFrame
ratings_summary_df.head()

,averagerating,numvotes,primary_title
0,8.3,31,Laiye Je Yaarian
1,8.9,559,Borderless
2,6.4,20,Just Inès
3,4.2,50352,The Legend of Hercules
4,6.5,21,Até Onde?


In [ ]:
# Finding out if there is a relationship between the rating and runtime in minutes
from scipy.stats import pearsonr

# Drop rows with missing values in 'averagerating' or 'runtime_minutes'
ratings_runtime_df = ratings_basics_merged[['averagerating', 'runtime_minutes']].dropna()

# Perform Pearson correlation test
correlation, p_value = pearsonr(ratings_runtime_df['averagerating'], ratings_runtime_df['runtime_minutes'])

# Display the results
print(f"Pearson Correlation Coefficient: {correlation}")
print(f"P-value: {p_value}")

# Interpret the results
if p_value < 0.05:
    print("There is a statistically significant relationship between rating and runtime.")
else:
    print("There is no statistically significant relationship between rating and runtime.")

Pearson Correlation Coefficient: -0.006963469265963981
P-value: 0.07311196545893413
There is no statistically significant relationship between rating and runtime.


This analysis provides valuable insights into whether the length of a movie—whether longer or shorter—is associated with higher or lower audience ratings, offering clues about viewer preferences. However, it's crucial to emphasize that a correlation between runtime and ratings does not necessarily indicate causation. Even if a strong statistical link would have been observed, it would not imply that the duration of a movie directly affects how audiences rate it.

In [6]:
# Loading data from rotten tomatoes
rotten_tomatoes_data = pd.read_csv('zippedData/rt.movie_info.tsv.gz', compression='gzip', sep='\t')
rotten_tomatoes_data.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


While loading the data, the parameters ```compression='gzip'``` was needful to handle decompressing the file when reading it and the parameter ```sep=\t``` which ensured the data is correctly parsed with tab-separated columns instead of the default comma-separated format(CSV)

In [7]:
rotten_tomatoes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


As you can see from the output above, this dataset contains several columns with information on different movies. Some of the information that could be of use to us would have been ```box_office```. However, after getting the information of the dataset, the column of interest has many missing values. This dataset therefore wouldn't be of use to answer the business questions. 

In [8]:
# Loading data from the movie database
the_movies_db = pd.read_csv("zippedData/tmdb.movies.csv.gz", compression='gzip')
the_movies_db.tail()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.6,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.6,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.6,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.6,2018-06-22,Trailer Made,0.0,1
26516,26516,"[53, 27]",309885,en,The Church,0.6,2018-10-05,The Church,0.0,1


In [22]:
the_movies_db.columns

Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')

To understand better the genres of the different movies, I looked up the movie genre ids from [The Movie Database](https://www.themoviedb.org) and found how to interpret them [Genre IDs](https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee.). Therefore, I am going to insert a column ```genre``` in the table to highlight which specific genre each movie is. 

In [39]:
# Mapping genre IDs to genre names using a dictionary
genre_dict = {
    28: "Action", 12: "Adventure", 16: "Animation", 35: "Comedy", 80: "Crime",
    99: "Documentary", 18: "Drama", 10751: "Family", 14: "Fantasy", 36: "History",
    27: "Horror", 10402: "Music", 9648: "Mystery", 10749: "Romance", 878: "Science Fiction",
    10770: "TV Movie", 53: "Thriller", 10752: "War", 37: "Western"
}

In [ ]:
# Convert genre IDs to genre names
import json
the_movies_db['genre_names'] = the_movies_db['genre_ids'].apply(
    lambda x: [genre_dict.get(genre_id, "Unknown") for genre_id in json.loads(x)]
)

# Display the updated DataFrame
the_movies_db[['genre_ids', 'genre_names']].head()

,genre_ids,genre_names
0,"[12, 14, 10751]","[Adventure, Fantasy, Family]"
1,"[14, 12, 16, 10751]","[Fantasy, Adventure, Animation, Family]"
2,"[12, 28, 878]","[Adventure, Action, Science Fiction]"
3,"[16, 35, 10751]","[Animation, Comedy, Family]"
4,"[28, 878, 12]","[Action, Science Fiction, Adventure]"


In [ ]:
# Sorting the_movies_db by highest popularity
the_movies_db.sort_values(by="popularity", ascending=False)


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,genre_names
23811,23811,"[12, 28, 14]",299536,en,Avengers: Infinity War,80.773,2018-04-27,Avengers: Infinity War,8.3,13948,"[Adventure, Action, Fantasy]"
11019,11019,"[28, 53]",245891,en,John Wick,78.123,2014-10-24,John Wick,7.2,10081,"[Action, Thriller]"
23812,23812,"[28, 12, 16, 878, 35]",324857,en,Spider-Man: Into the Spider-Verse,60.534,2018-12-14,Spider-Man: Into the Spider-Verse,8.4,4048,"[Action, Adventure, Animation, Science Fiction..."
11020,11020,"[28, 12, 14]",122917,en,The Hobbit: The Battle of the Five Armies,53.783,2014-12-17,The Hobbit: The Battle of the Five Armies,7.3,8392,"[Action, Adventure, Fantasy]"
5179,5179,"[878, 28, 12]",24428,en,The Avengers,50.289,2012-05-04,The Avengers,7.6,19673,"[Science Fiction, Action, Adventure]"


As you can see from the output above, this dataset contains several columns with information on different movies. Some of the columns are of interest and will be used to answer the business questions as will be seen. 

In [34]:
# Loading data from "the numbers website" 
movie_numbers = pd.read_csv("zippedData/tn.movie_budgets.csv.gz", compression='gzip')
movie_numbers.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


As you can see from the output above, this dataset contains several columns with information on different movies. The information includes gross of different movies, production budges and the release dates of the movies.  

In [35]:
merged_movies = movie_numbers.merge(the_movies_db, left_on='movie', right_on='title', how='inner')
merged_movies.head()

,id_x,release_date_x,movie,production_budget,domestic_gross,worldwide_gross,Unnamed: 0,genre_ids,id_y,original_language,original_title,popularity,release_date_y,title,vote_average,vote_count,genre_names
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",6,"[28, 12, 14, 878]",19995,en,Avatar,26.526,2009-12-18,Avatar,7.4,18676,"[Action, Adventure, Fantasy, Science Fiction]"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2470,"[12, 28, 14]",1865,en,Pirates of the Caribbean: On Stranger Tides,30.579,2011-05-20,Pirates of the Caribbean: On Stranger Tides,6.4,8571,"[Adventure, Action, Fantasy]"
2,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",14169,"[28, 12, 878]",99861,en,Avengers: Age of Ultron,44.383,2015-05-01,Avengers: Age of Ultron,7.3,13457,"[Action, Adventure, Science Fiction]"
3,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",23811,"[12, 28, 14]",299536,en,Avengers: Infinity War,80.773,2018-04-27,Avengers: Infinity War,8.3,13948,"[Adventure, Action, Fantasy]"
4,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209",20623,"[28, 12, 14, 878]",141052,en,Justice League,34.953,2017-11-17,Justice League,6.2,7510,"[Action, Adventure, Fantasy, Science Fiction]"


In [36]:
merged_movies.columns

Index(['id_x', 'release_date_x', 'movie', 'production_budget',
       'domestic_gross', 'worldwide_gross', 'Unnamed: 0', 'genre_ids', 'id_y',
       'original_language', 'original_title', 'popularity', 'release_date_y',
       'title', 'vote_average', 'vote_count', 'genre_names'],
      dtype='object')

In [38]:
selected_merged_movies = merged_movies[['movie', 'production_budget', 'domestic_gross', 'worldwide_gross', 'popularity', 'title', 'vote_average', 'genre_names']]
selected_merged_movies.head()

,movie,production_budget,domestic_gross,worldwide_gross,popularity,title,vote_average,genre_names
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",26.526,Avatar,7.4,"[Action, Adventure, Fantasy, Science Fiction]"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",30.579,Pirates of the Caribbean: On Stranger Tides,6.4,"[Adventure, Action, Fantasy]"
2,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",44.383,Avengers: Age of Ultron,7.3,"[Action, Adventure, Science Fiction]"
3,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",80.773,Avengers: Infinity War,8.3,"[Adventure, Action, Fantasy]"
4,Justice League,"$300,000,000","$229,024,295","$655,945,209",34.953,Justice League,6.2,"[Action, Adventure, Fantasy, Science Fiction]"
